## TDI Project Part 1: Drug utilization by drug name

Objective: Develop a dashboard app that enables us to browse through drugs and assess the level of prescription compared to other drugs in the same category over time.

Deliverables:
1. A method of displaying the level of prescription for every drug in a given category over the years, so that we can observe population-wide trends over time.

2. Ability to identify the most prescribed drugs based on the level of prescription, so that we know which drugs we are interested in focusing on for further study.

3. A method of visualizing trends for a specific drug.

Approach:
- We will use pandas module in Python to perform data manipulations for this app.
- We will use seaborn module in Python to visualize our data in a box-and-whisker plot.
- We will use a swarm plot overlay to identify outliers.
- We will implement several UI controls to allow ease of selecting our variables of interest. 
- We will use the Medicare Part D Prescriber dataset, a publicly available dataset.

This dataset is from the Center for Medicare and Medicaid Services website (https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Medicare-Provider-Charge-Data/Part-D-Prescriber.html). The shape of dataset from each year is as follows:

2013: (3447, 17)
2014: (3362, 17)
2015: (3395, 17)
2016: (3393, 19)

In [10]:
# import modules, read data and create a dict of data
%matplotlib inline
import seaborn as sns
import numpy as np
import pandas as pd
from ipywidgets import interact

data_2013=pd.read_csv('Part_D_Prescriber_National_Summary_Report__Calendar_Year_2013.csv')
data_2014=pd.read_csv('Part_D_Prescriber_National_Summary_Report__Calendar_Year_2014.csv')
data_2015=pd.read_csv('Part_D_Prescriber_National_Summary_Report__Calendar_Year_2015.csv')
data_2016=pd.read_csv('Part_D_Prescriber_National_Summary_Report__Calendar_Year_2016.csv')

data = {2013: data_2013,
        2014: data_2014,
        2015: data_2015,
        2016: data_2016}

In [11]:
print(data_2016.shape, data_2015.shape, data_2014.shape, data_2013.shape)

(3393, 19) (3395, 17) (3362, 17) (3447, 17)


In [12]:
# relabel columns for ease of use
col_mapper= {
    'Drug Name': 'drug_name',
    'Generic Name': 'generic_name',
    ' Number of Prescribers ': 'num_rx',
    ' Number of Medicare Part D Claims ': 'num_claims',
    ' Number of Standardized 30-Day Part D Fills ': 'num_30d_fills',
    'Aggregate Cost Paid for Part D Claims':'agg_cost_paid_claims',
    ' Number of Medicare Beneficiaries ': 'num_bene',
    ' GE65 Suppression Flag ': 'ge65_flag',
    ' Number of Medicare Part D Claims for Beneficiaries 65+ ': 'claims_65',
    ' Number of Standardized 30-Day Part D Fills for Beneficiaries 65+ ': 'num_30d_fills_65',
    'Aggregate Cost Paid for Part D Claims for Beneficiaries 65+': 'agg_paid_claims_65',
    ' Beneficiary 65+ Suppression Flag ': 'bene_65_flag',
    ' Number of Medicare Beneficiaries 65+ ': 'num_bene_65',
    'Aggregate Cost Share for Beneficiaries with Low Income Subsidy': 'agg_share_subsidy',
    'Aggregate Cost Share for Beneficiaries with No Low Income Subsidy': 'agg_share_no_subsidy',
    ' Opioid Drug Flag ': 'opioid',
    'Extended-Release Opioid Drug Flag': 'er_opioid',
    ' Antibiotic Drug Flag ':'abx',
    ' Antipsychotic Drug Flag ':'apsy',
    ' High Risk Medication (HRM) Drug Flag ':'hrm'
}
for y in [2013, 2014, 2015, 2016]:
    data[y] = data[y].rename(col_mapper, axis='columns')

In [15]:
# In this section, I created an interactive code to compare the ratio of prescribed drugs to beneficiaries over the years.
data_storage={}
year = [2013, 2014, 2015, 2016]

# function generates a swarmplot of a single drug in the observed category selected by the user
def g(drug_name):
    df_sum = pd.DataFrame()
    if drug_name == 'all':
        return
    else:
        # handles spacing issues in index labels
        for y in year:
            try:
                df_sum = pd.concat([df_sum, data_storage[y].loc[drug_name.strip()]], axis=1, sort=False)
            except:
                try:
                    df_sum = pd.concat([df_sum, data_storage[y].loc[drug_name.strip()+" "]], axis=1, sort=False)
                except:
                    df_sum
            finally:
                df_sum
        if len(df_sum)>0:
            df_sum = df_sum.transpose()
            ax = sns.swarmplot(x='year',y='ratio', data=df_sum,
                          color='red', size=6)
            ax.set_ylabel('Drugs-to-beneficiaries ratio')
            ax.set_xlabel('Year')
            ax.set_title('Drug supplied over time for {}'.format(drug_name))
        return

# function returns a table of drugs based on the observation threshold set by the user    
def h(threshold):
    df_sum = pd.DataFrame()
    for y in year:
        df_sum = pd.concat([df_sum, data_storage[y][data_storage[y]['ratio']>threshold]], axis=0, sort=False)
    return df_sum
    
# function generates a boxplot of a drug category selected by the user
def f(drug_type):
    d = {'Opioid': 'opioid', 'Antibiotics': 'abx', 'Antipsychotic': 'apsy', 'all':'all'}
    dx = d[drug_type]
    df_sum = pd.DataFrame()
    data_print=pd.DataFrame()
    for y in year:
        if dx == 'all':
            data_print = data[y]
        else:
            data_print = data[y][data[y][dx].str.strip()=='Y']
        data_print = data_print.pivot_table(index = 'generic_name', aggfunc = np.sum)
        data_print = data_print.dropna(axis=0)[data_print.num_bene>0]
        df = pd.DataFrame()
        df['ratio'] = data_print['num_30d_fills']/data_print['num_bene']
        df['year'] = y
        data_print=pd.concat([data_print,df],axis=1, sort=False)
        data_storage[y] = data_print.loc[:,['ratio','year']]
        df_sum = pd.concat([df_sum, df], axis=0, sort=False)
        
    sns.set(style='whitegrid')
    ax = sns.boxplot(x='year', y='ratio', data=df_sum, palette = 'vlag')
    ax = sns.swarmplot(x='year',y='ratio', data=df_sum,
                      color='black', size=5)
    ax.set_ylabel('Drugs-to-beneficiaries ratio')
    #ax.set_yscale('log')
    ax.set_xlabel('Year')
    ax.set_title('Drug supplied over time for {} class'.format(drug_type))
    
    ax = interact(g, drug_name=['all'] + list(data_print.index))
    interact(h, threshold = (0.0, df_sum.ratio.max()+1))
    return

interact(f, drug_type=['Opioid', 'Antibiotics', 'Antipsychotic']);

interactive(children=(Dropdown(description='drug_type', options=('Opioid', 'Antibiotics', 'Antipsychotic'), va…

Using the interactive dashboard above, we can make the following observations:

1. Among opioids, methadone, butorphanol tartrate and nalbuphine lead the category.

2. Among antibiotics, trimethoprim, demeclocycline and erythromycin lead the category.

3. Among antipsychotics, common medications such as clozapine (Clozaril) and risperidone microspheres (Risperdal) lead the category.

,num_rx,num_claims,num_30d_fills,agg_cost_paid_claims,num_bene,claims_65,num_30d_fills_65,agg_paid_claims_65,num_bene_65,agg_share_subsidy,agg_share_no_subsidy
count,3233.000000,3.393000e+03,3.393000e+03,3.393000e+03,3.236000e+03,3.082000e+03,3.082000e+03,3.082000e+03,2.784000e+03,3.393000e+03,3.393000e+03
mean,25748.654500,4.367964e+05,6.563077e+05,4.306785e+07,1.172238e+05,3.716120e+05,5.864185e+05,3.252470e+07,1.090286e+05,2.891249e+05,4.358735e+06
std,71271.002618,2.362425e+06,3.990124e+06,1.861502e+08,5.558343e+05,2.019011e+06,3.591753e+06,1.402510e+08,4.939454e+05,1.235104e+06,1.733599e+07
min,11.000000,1.100000e+01,1.100000e+01,1.122600e+02,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,145.000000,3.170000e+02,3.707000e+02,7.525630e+04,1.547500e+02,2.745000e+02,3.397750e+02,8.258295e+04,1.967500e+02,3.076000e+02,5.861810e+03
50%,1118.000000,3.423000e+03,4.316300e+03,1.176917e+06,1.263500e+03,2.947500e+03,3.706500e+03,1.102936e+06,1.436500e+03,4.427410e+03,9.704488e+04
75%,10382.000000,4.904600e+04,6.030690e+04,1.439267e+07,1.633325e+04,4.202175e+04,5.424095e+04,1.153013e+07,1.836175e+04,5.409299e+04,1.264731e+06
max,622206.000000,4.447331e+07,8.483562e+07,4.398534e+09,9.405696e+06,3.817320e+07,7.501537e+07,2.359691e+09,8.187209e+06,1.996467e+07,2.649509e+08
